In [1]:
import os
import shutil
import warnings
import urllib.request as request
from contextlib import closing

import cupy

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

from nvidia.cheminformatics.chembldata import ChEmblData

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [2]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=10000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().compute().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)
        cdf = cdf.compute()

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    

def recluster(tcdf):
    """
    Recluster on a dataset
    """
    tcdf = tcdf.persist()

    print('Reclustering...')
    tcdf = tcdf.drop(['x', 'y', 'cluster'], axis=1)

    print('Computing KMEANS...')
    kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
    kmeans_float.fit(tcdf)
    kmeans_labels = kmeans_float.predict(tcdf)

    print('Computing UMAP...')
    local_model = cuUMAP()
    X_train = tcdf.compute()
    local_model.fit(X_train)

    umap_model = cuDaskUMAP(local_model,
                            n_neighbors=n_neighbors,
                            a=1.0,
                            b=1.0,
                            learning_rate=1.0,
                            client=client)
    Xt = umap_model.transform(tcdf)

    # Add back the column required for plotting and to correlating data
    # between re-clustering
    tcdf['x'] = Xt[0]
    tcdf['y'] = Xt[1]
    tcdf['cluster'] = kmeans_labels
    
    return tcdf

# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:37035 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 3 Cores: 3 Memory: 33.58 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [10]:
%%time
cache_directory = '/data/fp'

if cache_directory is None:
    chem_data = ChEmblData(db_file=db_file)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()

# Reduce
pca = cuDaskPCA(client=client, n_components=pca_comps)
dcudf = pca.fit_transform(dcudf)

CPU times: user 11.5 s, sys: 347 ms, total: 11.9 s
Wall time: 18.7 s


# Clustering

In [11]:
%%time

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(dcudf)
kmeans_labels = kmeans_float.predict(dcudf)

CPU times: user 98 ms, sys: 16 ms, total: 114 ms
Wall time: 356 ms


In [12]:
%%time
local_model = cuUMAP()
X_train = dcudf.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(dcudf)

dcudf['x'] = Xt[0]
dcudf['y'] = Xt[1]
dcudf['cluster'] = kmeans_labels

dcudf.head(5)

CPU times: user 389 ms, sys: 173 ms, total: 562 ms
Wall time: 1.1 s


,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,x,y,cluster
0,-1.287431,-0.559803,0.599178,0.651916,1.173030,0.009061,1.197854,-0.641295,-0.716238,-0.053472,...,0.350103,-0.488843,-0.009733,0.067950,-0.427004,0.579675,-0.158319,-4.000153,9.777552,2
1,-1.215974,-0.083825,0.278043,0.028548,1.434533,-0.497732,0.943763,-0.108586,-0.380386,-0.541477,...,0.933025,-0.517051,0.427014,0.017982,-0.543311,0.405936,-0.239332,-4.040936,9.742472,2
2,-1.262150,-0.424553,0.518995,0.021030,1.525846,0.973738,0.222641,-0.843903,-0.212779,-0.263492,...,0.173930,0.133231,0.237898,-0.072275,0.071529,0.385734,-0.070083,-4.068999,9.723571,2
3,-1.222840,-0.031298,0.678749,-0.009444,1.018124,-0.497639,0.778538,-0.036022,-0.659397,-0.638758,...,0.384355,0.168377,0.235355,-0.261249,-0.423860,0.147132,0.243990,-4.024139,9.751870,2
4,-1.319906,-0.270999,0.574376,-0.021628,1.818873,0.636202,1.339735,-0.811215,-0.699236,-0.104649,...,0.264651,-0.158958,0.238779,-0.042836,-0.179382,0.390000,-0.196633,-4.010470,9.752064,2


In [13]:
molregno = dcudf.index
dcudf['id_exists'] = dcudf.index.isin([2, 4])

df = dcudf.query('id_exists == True')
df = df.compute()

In [17]:
df.index.to_array()

array([2, 4])

In [ ]:
%%time
show_cluster_plot(dcudf)

# Re-clustering

In [ ]:
%%time
# Filter data before reclustering
cluster = 1                                         # <- PLEASE CHANGE CLUSTER AS REQUIRED
query = 'cluster == ' + str(cluster)
tcdf = dcudf.query(query)

show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))
tcdf = recluster(tcdf)
print('Plotting UMAP...')
show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))